RESTART kernel prior to running after any changes to config files...

### import configuration and functions modules

In [1]:
%%time
import config as cf
import functions as f
from list_builder import prepare_master_list, build_list

CPU times: user 448 ms, sys: 40 ms, total: 488 ms
Wall time: 611 ms


### build pkl files
master, fur, sg(special group), active_each_month, order proposals, month percentage

In [2]:
%%time
%run build_program_files

CPU times: user 1.48 s, sys: 12 ms, total: 1.49 s
Wall time: 1.5 s


### build compensation tables
convert from input Excel file, detects config sample_mode if selected

In [4]:
%%time
%run make_pay_tables_from_excel

CPU times: user 128 ms, sys: 0 ns, total: 128 ms
Wall time: 126 ms


### skeleton

In [5]:
%%time
%run make_skeleton master

CPU times: user 1.28 s, sys: 180 ms, total: 1.46 s
Wall time: 1.62 s


### standalone
standalone must be rerun after job level model change...

In [6]:
%%time
%run standalone prex

CPU times: user 2.34 s, sys: 412 ms, total: 2.76 s
Wall time: 2.87 s


### calculate for each list order
(including list conditions, job count changes, and possible furlough recall schedule)

In [7]:
%%time
%run compute_measures p1 prex ratio

CPU times: user 1.65 s, sys: 352 ms, total: 2 s
Wall time: 2.13 s


In [8]:
%%time
%run compute_measures p2 prex count

CPU times: user 1.85 s, sys: 352 ms, total: 2.2 s
Wall time: 2.54 s


In [9]:
%%time
%run compute_measures p3 prex

CPU times: user 1.76 s, sys: 300 ms, total: 2.06 s
Wall time: 2.25 s


In [ ]:
%%time
#%run compute_measures award prex count

### merge an ordered list with the master list

In [10]:
%%time
%run join_inactives master p3 final ffill

            empkey  eg    lname        dob        doh      ldate  sg  fur  \
empkey                                                                      
10011102  10011102   1  tooeyoo 1949-07-13 1973-02-26 1975-01-29   0    0   
10010475  10010475   1  rubelot 1949-02-05 1975-05-27 1975-05-27   0    0   
10013096  10013096   1  yeloxid 1949-01-08 1977-01-18 1977-01-18   0    0   
10012178  10012178   1  xayeaue 1951-06-07 1977-11-15 1977-11-15   0    0   
10014447  10014447   1  finuceu 1951-10-17 1977-12-09 1977-12-09   0    0   

          line  eg_order    retdate  idx  
empkey                                    
10011102     1         1 2014-07-13    1  
10010475     1         2 2014-02-05    2  
10013096     1         3 2014-01-08    3  
10012178     1         4 2016-06-07    6  
10014447     1         5 2016-10-17    7  
CPU times: user 2.16 s, sys: 8 ms, total: 2.16 s
Wall time: 2.17 s


In [11]:
%%time
f.print_config_selections()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.35 ms


,setting
option,
annual_pcnt_raise,0.02
case_study,sample3
compute_job_category_order,True
compute_pay_measures,True
compute_with_job_changes,True
delayed_implementation,True
discount_longev_for_fur,True
enhanced_jobs,True
full_time_pcnt1,0.6


### cells below for demonstration only

### build a 'hybrid' list example:
(stored as 'dill/hybrid.pkl')

In [12]:
%%time
# this will build a "hybrid" list with equal weighting given to longevity and standalone job position
master = prepare_master_list()
build_list(master, ['ldate', 'jobp'], [.5, .5])

CPU times: user 168 ms, sys: 12 ms, total: 180 ms
Wall time: 177 ms


In [13]:
%%time
# generate dataset for the hybrid list
%run compute_measures hybrid prex

CPU times: user 1.63 s, sys: 368 ms, total: 2 s
Wall time: 2.13 s


### sample of files created

In [14]:
%%time
# print only the first 5 lines of each file
file_names = ['master', 'skeleton', 'standalone', 'p_p1', 'pay_table_enhanced', 'ds_p1', 'p_hybrid', cf.case_study + '_final']
for file in file_names:
    df = pd.read_pickle('dill/' + file + '.pkl')
    print('< ' + file.upper() + ' >', '\n', 'row count: ' + str(len(df)),
        '\n', 'column count: '+ str(len(df.columns)), '\n',
        df.columns, '\n',
        '\n', df.head().to_string(line_width=120), '\n\n', '-' * 80, '\n')

< MASTER > 
 row count: 7510 
 column count: 11 
 Index(['empkey', 'eg', 'lname', 'dob', 'doh', 'ldate', 'sg', 'fur', 'line',
       'eg_order', 'retdate'],
      dtype='object') 
 
             empkey  eg    lname        dob        doh      ldate  sg  fur  line  eg_order    retdate
empkey                                                                                              
10011102  10011102   1  tooeyoo 1949-07-13 1973-02-26 1975-01-29   0    0     1         1 2014-07-13
10010475  10010475   1  rubelot 1949-02-05 1975-05-27 1975-05-27   0    0     1         2 2014-02-05
10013096  10013096   1  yeloxid 1949-01-08 1977-01-18 1977-01-18   0    0     1         3 2014-01-08
10012178  10012178   1  xayeaue 1951-06-07 1977-11-15 1977-11-15   0    0     1         4 2016-06-07
10014447  10014447   1  finuceu 1951-10-17 1977-12-09 1977-12-09   0    0     1         5 2016-10-17 

 -------------------------------------------------------------------------------- 

< SKELETON > 
 row count